<a href="https://colab.research.google.com/github/knobs-dials/wetsuite-dev/blob/main/notebooks/intermediate/technical_localdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Purpose of this notebook

An introduction to storing collections of things in the localdata module, mostly the LocalKV class, that we provide.

## tl;dr for the programmers

It's a key-value store that persists in a file, with some self-enforced type checking. 
Try not to do read and write concurrently.

In [12]:
import wetsuite.helpers.localdata

# You can open one like:
mystore = wetsuite.helpers.localdata.LocalKV('example_deleteme.db', str,str)
# and then use it like:
mystore.put('foo', 'bar')
mystore.get('foo')
#mystore.delete('foo')
display( list( mystore.items()) )     # Gives [('foo', 'bar')]


# There is a helper that lets you do HTTP fetches into a str-to-bytes store, like:
fetch_store = wetsuite.helpers.localdata.LocalKV('fetch_deleteme.db', str,bytes)
wetsuite.helpers.localdata.cached_fetch(fetch_store, 'http://example.com')
display( list(fetch_store.items()))   # Gives [('http://example.com', b'<!doctype html>\n<html>...


# There is also a variant that lets you store python structures (of types that msgpack can encode)
d_store = wetsuite.helpers.localdata.MsgpackKV('dicts_deleteme.db', str)
d_store.put('foo', {'bar':1})
display( list(d_store.items()))       # Gives [('foo', {'bar': 1})]

[('foo', 'bar')]

[('http://example.com',
  b'<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n        div {\n            margin: 0 auto;\n            width: auto;\n        }\n    }\n    </style>    \n</he

[('foo', {'bar': 1})]

## The longer introduction

**Why?**

Datasets that consist of files that someone cureated are
nicely portable, but not the fastest to deal with in terms of writing code (consider also filename limits) _or_ opening all that all the time.
More project than you'ld think float on "flat collection of things", whether that's files or S3 buckets.


Creating intermediate data (also e.g. to avoid double work) has similar bookkeeping and efficiency issues (have enough files and just walking through them takes minutes, before you've started any work).

You can do all that in just files, and if you're comfortable with that, have at it,
but there is clunkiness and inefficiency you cannot avoid.


On the other end you can have a structured database. 
They may be efficient and convenient, but only aftter a lot of up-front work - data design, installing it, etc.

We might like an inbetween.


***We've had computers for decades. Isn't this a solved problem?***

For a good part, yes -- in different ways in different fields and to different degrees.

Say, HDF5 is a great way to deal with large amounts of structured numberic data.

More recently, Arrow is a file format that is quick to read, and reasonable to parallelize.
...but not so much to write.


Also, neither is _quite_ for text or for schemaless data.

Also, they require a library, though not a seprate service, which would be okay if they check all other boxes - but they don't.

***At the risk of committing a "not invented here", maybe we'll roll our own.  What are our wishes?***

It would be useful to have a mix of
- saving things to disk, so that it sticks around
- easy to use - loading and saving in a few lines of code at most
- easy to find specific items you know you have
- allows storing text, arbitrary files (probably as bytestrings), and _perhaps_ more
- working with moderately large data
    - allows iteration of large data (without loading all in RAM)
    - allows _reasonably_ fast lookup of arbitrary items (without loading all in RAM or scanning the entire file)
- preferably not requiring a library you need to install, and/or a database you need to install (both potentially requiring an admin)


***Do we need all that?***

No. 

More specifically, only sometimes, 
yet if we present a single thing for multiple needs, we want to check more boxes.

So we made something that fits both "can store intermediate data" and "can use to distribute datasets" description.

(It's a thin wrapper around sqlite3 -- but ideally you don't need to know about that. It's mentioned because this is part of the python3 standard library, meaning that if you use python3 - and you very likely do -  you need no further install to use this).

There are still plenty of limitations. 

In particular, you can't access things concurrently (yes, WAL allows more concurrency, but it's less portable).

In [1]:
import pprint, datetime
import wetsuite.helpers.localdata

### basic key-value use

Not very useful yet, but a minimal example:

In [2]:
# given that we open a store like:
mystore = wetsuite.helpers.localdata.LocalKV('mystore.db', key_type=str,value_type=str)   # we tell it the keys should be string values, and so should the values

# we can do:
mystore.put('fish', 'one')
mystore.put('fork', 'test')
mystore.get('fish')
mystore.delete('fish')

In [3]:
# There are also some functions like those present on python dicts, like
print( 'keys:                    ', list(mystore.keys())    )   # list() because we we actually get a view   (much like python 3 dict behaviour)
print( 'values:                  ', list(mystore.values())  ) 
print( 'items:                   ', list(mystore.items())   )
print( 'len:                     ', len(mystore)      )
print( '"fork" is a present key: ', ('fork' in mystore) )       # via __contains__()

keys:                     ['a', 'fork']
values:                   ['b', 'test']
items:                    [('a', 'b'), ('fork', 'test')]
len:                      2
"fork" is a present key:  True


## Why this can be useful for files, and downloading

Say you download a bunch of webpages, or gigabytes of PDFs. 

Yes, storing that into separate files is perfectly functional. 
The filesystem _is_ a simple database, after all.

And it checks _most_ of the boxes mentioned above - except ease of use / few lines of code,
because there is a bunch of boilerplate to write each use.

And if you start doing this seriously, you will need to start thinking about 
- _not_ hardcoding paths of where you've actually put it,
- what you name each on the filesystem (because most filesystems won't allow filenames longer than 255, and many URLs are),
- finding specific items, based on the renaming you did, or maybe based on creating your own index
and more such details.

Sometimes none of that is hard, things just work. 
And being able to directly open the files for inspection is great.

Yet in other cases all those details means this becomes a messy amount of extra work.
And maybe you'll have to do that for each project, or each dataset.


So we offer a disk-backed [key-value store](https://en.wikipedia.org/wiki/Key%E2%80%93value_database)
(meaning each unique key is associated with a value - and no more structure than that)
that lets you easily persist certain kinds of data, from relatively brief code.

It's fairly minimal, but that's part of the point. 

### Downloading into a store

While collecting data online, we tend to download a multitude of items.

If you have a bunch of URLs, want to fetch the according documents,
maybe without putting each into 
and want to cache locally to avoid re-fetching on each run of a script,
then our `cached_fetch( store, url )` can help:

If we have previously fetched that URL, we can return the stored copy. If we did not, we fetch it, save it, then return it.
<!-- (...assuming you don't need auth or other session state, and assuming you don't need them removed based on time and/or cacheing rules. We may fix some of that later) -->

In [4]:
download_store = wetsuite.helpers.localdata.LocalKV('download_store.db', key_type=str, value_type=bytes)  # values should be bytes, because cached_fetch stores downloads while still in bytestring form

page_bytes, came_from_cache = wetsuite.helpers.localdata.cached_fetch( download_store, 'http://example.com/' )
print( f'Came from cache: {came_from_cache}\n' )
print( page_bytes.decode('u8') )

Came from cache: True

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain

## On the name / path you want it to open

tl;dr:
- a bare filename means you get the same store each time, no matter what, in your homedir.
- anything else, that looks absolute (or relative) gets handed through as-is.

For context, handing a pathless base name to bare sqlite would just put it in the current directory.

The `cwd` isn't always where you think, so this is likely to sprinkle databases all over the place.
This is a fairly confusion/mistake around sqlite - and files in general.

Yo make it a little more controlled where things go, and avoid you you focusing on the not-very-portable workaround of using absolute paths, LocalKV and derived constructors will...
- Given a **bare name**, e.g. `extracted.db`, returns an absolute path 
    within a "this is where wetsuite keeps its stores directory" within your user profile,
    e.g. /home/myuser/.wetsuite/stores/extracted.db or C:\Users\myuser\AppData\Roaming\.wetsuite\stores\extracted.db
    Most of the point is that handing in the same name will lead to opening the same store, regardless of details.

- given `:memory:` will create a memory-only store, not backed by disk

- given an **absolute path**, will pass that through, and do what you expect
  - this is not very portable if you do it directly
  - ...yet this behaviour is useful in other ways: if you actually _wanted_ it in the current directory, instead of this function 
    consider something like  `os.path.abspath('mystore.db')`

- given a **relative path**, it will pass that through -- which will open it relative to the current directory

This is actually a description of what `wetsuite.helpers.localstore.resolve_path()` does. 
That helper just happens to be called in each constructor.


In [ ]:
print( wetsuite.helpers.localdata.resolve_path('test.db') )
print( wetsuite.helpers.localdata.resolve_path('rel/test.db') )
print( wetsuite.helpers.localdata.resolve_path('/tmp/abs/test.db') )
print( wetsuite.helpers.localdata.resolve_path(':memory:') )

In [ ]:
# The KV functions also call this internally, so you can also see that it _actually_ put it there:
store = wetsuite.helpers.localdata.LocalKV('deleteme.db', key_type=str, value_type=str)
store.path

## Note on typing

What types of python objects could we store in there? 

So, sqlite3 is a [DB-API](https://peps.python.org/pep-0249/)-like thing, 
so in theory we could hand in various python types - but will they be that same type once they are stored and fetched out again?

Let's find out.

In [5]:
typestore = wetsuite.helpers.localdata.LocalKV('typetest.db', str,None)   # keys:strings,   values: not checking so it does whatever sqlite decides

typestore.put('one', 1)
typestore.put('two', 2.2)
typestore.put('three', '3')
typestore.put('four', b'4')
typestore.put('five', datetime.datetime.now())

pprint.pprint( list( typestore.items() ) )

[('five', '2023-11-05 12:59:03.691843'),
 ('four', b'4'),
 ('one', '1'),
 ('three', '3'),
 ('two', '2.2')]


If you peek into the database from a shell using `sqlite3 typetst.db`, then `SELECT key, value, typeof(value) FROM kv`,  it is revealed that's all `text` except for one `blob`

tl;dr for the users: 
- Whenever your use is served by storing only `bytes` (e.g. for downloaded files) or `str` (any text), sticking to that keeps things simple
- We can do more, but that gets confusing  (see the rest of this cell why)
- If you want to store more complex things, particularly nested python data structures, see the rest of this notebook


To the programmers: 
- SQLite is [dynamically typed](https://en.wikipedia.org/wiki/Dynamic_typing), so will
  - store whatever type it decides to store, without adhering to the type you have declared for the column
    - and your language's sqlite library may have a say in that too
  - return the type that got stored, without converting
    - and your language's sqlite library may have a say in that too

- python's sqlite3 library already seems to help [coercing](https://en.wikipedia.org/wiki/Type_conversion) -- _roughly_: to `str` unless it's `bytes`

This combination makes it easy to mix things and confuse yourself.


That potential confision is why, when opening a store, 
we ask you to decide up front what type to put in, 
and why it actively checks, as a sanity check that _we_, 
the programmer, are actually being consistent.

(You _can_ disable that checking by handing in None for both.)

Caveat: the types it checks aren't stored (yet?),
so you better be consistent about opening the same store with the same type checking later.

# Serializing more complex data.


You will often end up creating structured data, at first in the form of lists and such.
: Writing as CSV works until you need commas, which tends to be too soon
: writing TSV works until your data accidentally contains tabs. CSV and TSV are common because they're just so easy to read and write and work well enough.

And then you may want a bit of nested types and/or mixed types e.g. lists, dicts; of ints, of floats, strings, bytes.


Writing JSON files is a good option actually, also because it has become and informal standard for a lot of interchange.
One-JSON-thing-per line files (often called JSONL, sometims NDJSON, LDJSON) also avoid the many-file problem.
I would generally recommend this for portable interchange, when you want other people to pick up your data.


SQLite has no reason to natively understand whatever python is doing, so if we want it to later return what we handed in, then it is up to our middle later to pack it into something that does that, and unpack it on use.

Options include.
- Python's pickle - stores [more than just the primitives](https://docs.python.org/3/library/pickle.html#what-can-be-pickled-and-unpickled), though has some security concerns
- JSON - a common interchange format, though it doesn't have a type for `date`/`datetime`
- MsgPack - faster than JSON, though doesn't do datetime out of the box either

Each has their own advantages and drawbacks. 

We chose to focus on `msgpack`, so there is a `MsgpackKV`, which handles the same as LocalKV but with that mentioned extra handling (implementation-wise, they just add an encode on `put()`, and decode on `get()`). 
Consider:

In [6]:
msgpack_store = wetsuite.helpers.localdata.MsgpackKV('msgpacktest.db')
msgpack_store.put('one', {'a':['b']})
print( msgpack_store.get('one') )
type( msgpack_store.get('one') ) # the type of what comes out is...   (mostly to point out it's not that as text)

{'a': ['b']}


dict

In [21]:
# ...and yes, you _could_ just use that always, even for more primitive types.
#   but maybe don't do that unless you're fine with mixing types in the way that LocalKV prevents
msgpack_store.put('two', 'three')
type( msgpack_store.get('two') )

for k, v in msgpack_store.items():
    print( '%s:%s  %s:%s'%(k, type(k).__name__, v,type(v).__name__) )


one:str  {'a': ['b']}:dict
two:str  three:str


# More notes

## Limitations

...you should probably not be using this expecting speed or parallelism.

We default to commit immediately (what we call autocommit), so for small things are limited by IOPS. 
You can put() with commit=False and eventually commit(), which on bulk inserts is much faster,
but it's slightly riskier in that if that borks, you may have to do some manual cleanup.

Parallelism is limited, because sqlite's is - multiple processes can read, but only one can write.

Also, assume this This is basically an append-only database.
If you remove items, assume it will fragment and that the file stays large,
until you do a vacuum, which basically rewrites the entire file.

## Minimal syntax hackery
If you want to store something, you always use `put()`. If you want to getch something, use `get()`.

Sure, we could save you a few keystrokes by making it act like a dict,
but that starts to be a leaky abstraction particularly once we get to storing more complex values -
if you expected ORM-like behaviour where altering those might get mirrored in the database, that often wouldn't work and arguably shouldn't (for hard-to-explain reasons). 

So we make you do it, to keep things clear. 